In [5]:
from imports import *
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np

In [4]:
# Code to install webdriver if it's not installed
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())

ModuleNotFoundError: No module named 'webdriver_manager'

In [157]:
# Log into linkedin
def login(driver, password):
    url =  'https://www.linkedin.com/checkpoint/rm/sign-in-another-account?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin'
    wait = WebDriverWait(driver, 10)
    driver.get(url)
  
    driver.find_element("xpath",'/html/body/div/main/div[2]/div[1]/form/div[1]/input').send_keys('wang.meredith09@gmail.com')
    driver.find_element("xpath",'/html/body/div/main/div[2]/div[1]/form/div[2]/input').send_keys(password)
    driver.find_element("xpath",'/html/body/div/main/div[2]/div[1]/form/div[3]').click()

# Redirects driver to the query link
def search(driver):
    time.sleep(5)
    driver.get("https://www.linkedin.com/jobs/search/?keywords=data%20science")

# Grab the number of results that were fetched
def get_n_results(driver):
    time.sleep(10)
    results_div = driver.find_element("xpath",'/html/body/div[4]/div[3]/div[4]/div/div/main/div/section[1]/header/div[1]/small')
    n_string = results_div.text
    n = int(n_string.split()[0].replace(',',""))
    return n 

# Finds job ul div
def get_jobs(driver):
    ul_div = driver.find_element("xpath",'/html/body/div[4]/div[3]/div[4]/div/div/main/div/section[1]/div/ul')
    return ul_div


# Scrolls to properly load page
def scroll_down(driver):
    for i in np.linspace(0,1,10):
        time.sleep(2)
        # Excute JavaScript
        # driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight*{i})")

# Getting each job's url, role name, company, and location
def get_job_urls(jobs,driver,job_urls = {}):
    i = 1
    while True: 
        try:

            WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH,f'/html/body/div[4]/div[3]/div[4]/div/div/main/div/section[1]/div/ul/li[{i}]')))
            url = jobs.find_element("xpath",f'/html/body/div[4]/div[3]/div[4]/div/div/main/div/section[1]/div/ul/li[{i}]/div/div[1]/div[1]/div[2]/div[1]/a').get_attribute("href")
            role = jobs.find_element("xpath",f'/html/body/div[4]/div[3]/div[4]/div/div/main/div/section[1]/div/ul/li[{i}]/div/div[1]/div[1]/div[2]/div[1]/a').text
            company = jobs.find_element("xpath",f'/html/body/div[4]/div[3]/div[4]/div/div/main/div/section[1]/div/ul/li[{i}]/div/div[1]/div[1]/div[2]/div[2]/a').text
            location = driver.find_element("xpath",f'/html/body/div[4]/div[3]/div[4]/div/div/main/div/section[1]/div/ul/li[{i}]/div/div[1]/div[1]/div[2]/div[3]/ul/li').text
            job_urls.update({url:{'company':company,'location':location,'role':role}})
            i+=1
        except:
            return job_urls

# Click on the button to go to the next page
def load_next_page(driver):
    curr= driver.find_element("xpath",'//*[@aria-current="true"]').text
    next = driver.find_element("xpath",f'//*[@aria-label="Page {int(curr)+1}"]')
    next.click()

# Use the urls to further get job's description
def get_description(driver,job_dict):
    good = []
    fail = []
  #Iterate through the url list to scrape the descriptions
    for url in list(job_dict.keys()):
        if url not in good:
            try:
                driver.get(url)
                time.sleep(3)
                if driver.current_url != url:
                    print(f'failed at {url}')
                    #remove broken urls
                    job_dict.pop(url)
                #scrape
                driver.find_element("xpath",'//*[@aria-label="Click to see more description"]').click()
                WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[2]/article/div/div[1]')))
                description = driver.find_element("xpath",'/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[2]/article/div/div[1]').text
                education = driver.find_element("xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[6]/div/div[1]/div[3]/div[2]/ul').text

                # time.sleep(3)
                job_dict.get(url).update({"description":description})
                job_dict.get(url).update({"education": education})
                time.sleep(6)
                good.append(url)

            except:
                #keep going if there is a random error in which a div did not load properly but check where we failed
                print(f"fail {job_dict.get(url)}")
                fail.append(url)
    return job_dict

# Get job's applicants' most have skills in the LinkedIn premium insight
# It will return the skills depending on if the user is LinkedIn premium
# This function is not used in the final main function
def get_skills(driver,job_dict):
    good = []
    fail = []
    # Iterate through the url list to scrape the descriptions
    for url in list(job_dict.keys()):
        if url not in good:
            try:
                driver.get(url)
                time.sleep(2)
                if driver.current_url != url:
                    print(f'failed at {url}')
                    #remove broken urls
                    job_dict.pop(url)
                # Scrape
                skill_list = driver.find_element("xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[6]/div/div[1]/div[2]/div[2]/ul')
                skills = skill_list.find_elements(By.TAG_NAME, "li")
                # if len(skills) > 0:
                # i = 1
                skill = []
                # while True:
                #     try:
                #         # WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH,f'/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[6]/div/div[1]/div[2]/div[2]/ul/li[{i}]')))
                #         skillname = driver.find_element("xpath",f'/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[6]/div/div[1]/div[2]/div[2]/ul/li[{i}]/p').text
                #         job_dict.get(url).update({"skills":skillname})
                #         i+=1
                #         good.append(url)
                #     except:
                #         return job_dict
                for i in skills:
                    skill = i.find_elements("xpath", f'//*[@id="ember95"]/div/div[1]/div[2]/div[2]/ul/li[{i}]/p').text
                    job_dict.get(url).update({"skills":skill})
                    good.append(url)
                return job_dict
            except:
                #keep going if there is a random error in which a div did not load properly but check where we failed
                print(f"fail{job_dict.get(url)}")
                fail.append(url)

In [158]:
# Putting all the steps above together
def main(driver,password):
    login(driver, password)
    search(driver)
    n = get_n_results(driver)
    job_dict = {}
    #iterate through the amount of pages given
    for i in range(39):
        jobs = get_jobs(driver)
        scroll_down(driver)
        get_job_urls(jobs,driver,job_urls = job_dict)
        time.sleep(1)
        load_next_page(driver)
    get_description(driver,job_dict)
    df = pd.DataFrame(job_dict)
    # return get_description(driver,job_dict,good)
    return df

In [109]:
# Getting password
secret_file = open("env.txt", "r")
password = secret_file.read()
type(password)

str

In [159]:
# Calling the function and store job_dict into dataframe
driver = webdriver.Chrome('/Users/williambaldridge/linkedin-data-science-project/chromedriver')
secret_file = open("env.txt", "r")
password = secret_file.read()
newdf2 = main(driver,password)

# driver.quit()

fail {'company': 'Gallagher Bassett', 'location': 'Rolling Meadows, IL', 'role': 'Data Science'}
fail {'company': 'Procter & Gamble', 'location': 'Mason, OH', 'role': 'R&D Data Science'}
fail {'company': 'Bankrate', 'location': 'New York City Metropolitan Area', 'role': 'Product Manager, Data Science | Bankrate'}
fail {'company': 'Expedia Group', 'location': 'Austin, TX', 'role': 'Data Scientist III, Product Analytics'}
fail {'company': 'SoftStandard Solutions', 'location': 'United States', 'role': 'Data science'}
fail {'company': 'OMP', 'location': 'Atlanta, GA', 'role': 'Data Science Consultant'}
fail {'company': 'Booz Allen Hamilton', 'location': 'Arlington, VA', 'role': 'Data Scientist, Mid'}
fail {'company': 'Booz Allen Hamilton', 'location': 'Arlington, VA', 'role': 'Data Scientist, Mid'}
fail {'company': 'Booz Allen Hamilton', 'location': 'Arlington, VA', 'role': 'Data Scientist, Mid'}
fail {'company': 'Signet Jewelers', 'location': 'United States', 'role': 'Data Scientist'}
fai

In [162]:
# 2nd run - Checking result dataframe null values
newdf2.T.isnull().sum()

company          0
location         0
role             0
description    149
education      149
dtype: int64

In [153]:
# 1st run result dataframe
newdf

,https://www.linkedin.com/jobs/view/3209804028/?eBP=CwEAAAGCooswLC92xAgVhLwAfAp8BDV-bp9IgI0l_jjkwIHtqFIkmOKPW8OTbtMpt0jkXfnWCekOEUXtTFlAlr2v7oivnWmRp4s-2EMyS1FSi_3rt4K7SI7Y6zztKel1t8ASplWSfDCJkxbvVndefl-WLaNP1YcesHTLerOEnXOtgvE-T-oEKhg0k-SAHn9aRufCMeTiEXK5j91bw-7S3ZoSF2EC5zoMXQea0ZXtXFqwic5MNVZIVXyroKDyHzsB4OAXFgBmNcr62MwOitbpfVg4xcNcezAH8496btVXWOZZH1lSy_XkCL5eFvkk9UNVptlOKYACHRafLi1__jCKfo-A61yUuP3K-xNFY_EpqqCC644VGepXO3udXX7_Puex&recommendedFlavor=PRE_SCREENING_QUESTIONS&refId=lC3fZ8%2BOil0oPKKL03koLA%3D%3D&trackingId=CNb4E7RR3gP%2BP6pEnFZ1LA%3D%3D&trk=flagship3_search_srp_jobs,https://www.linkedin.com/jobs/view/3218989416/?eBP=CwEAAAGCooswLFPrId1XFpMvVIJ4yCJM4uTz0jxEWV5XVj519klP_nD7-Ee5TvFy795Psm801j06cFrjR5oGqJNuBOZxFSyYK5yACpW4dIAzTPWawqDbRGSINzkzqN4uommozbbc_WAtwwdU-VmlLtJDVNmqFLj5sAtlhXf90guNmpjYoAyIMwle5Ega5M8Bw79g_61w0-p5iTHuOU61ScmR95BAVEPOKFvLywF56zmkchRDYGAa4YssvO24ZBSijSBTtKH7hFxt1w52KrA8-1JRjvzHdPeTwmdT5GgbzcYNb0b1IejSlWhIJUUI_GIQFDMOHOP7TEmO-QrhTxAJClJVghNE-qtZEa0j7cLlddDQDrjU-OVfcrW-AeM-0XvH&recommendedFlavor=SKILL_ASSESSMENTS&refId=lC3fZ8%2BOil0oPKKL03koLA%3D%3D&trackingId=xdsGk07lEvrgg3rD601Kwg%3D%3D&trk=flagship3_search_srp_jobs,https://www.linkedin.com/jobs/view/3142891690/?eBP=CwEAAAGCooswLH_bUODSChzlZscRDJutgpFVlUTc5T4ZaQsI7vZkFrzEjRqtm-PstLwetV6-SlEWi0EaZCoVqz7mF3X_8h3Rqr4yqdMQHdOATepc2-nowV5kKjs9WIyySB-IuaopmwJzJBki3EvtvEo8PxXAfouYK-nMNWoXxaiP_l_R4zwXwXKcoKhdmtAzJdLvOPsLgREe-rd1MazHLBjT20g8VfGJg_5lI9zappJ5CVpym6N43O_fGUGUHp45nAwL91b7qfbjYI7NbJWkI2fJPTTRNVcr6NIYzHbpXCjAA62DDCM-3G5swAl6eUmeWax7KaBKnIvWbNxLMMefce1lCiSc0XSfzfBn9dyjAWTperwS1TBou56RpHH5g9XfkmI&recommendedFlavor=SKILL_ASSESSMENTS&refId=lC3fZ8%2BOil0oPKKL03koLA%3D%3D&trackingId=5D519cIlEyA4sPwZgBYlWQ%3D%3D&trk=flagship3_search_srp_jobs,https://www.linkedin.com/jobs/view/3077903474/?eBP=CwEAAAGCooswLO5XlkPlnOVXaF0SSeezj9PQILJrj4xYG05nSE0WStYxlHNy-NflpCNZBO5LMmERzWO5M41jmgU_kf3GuhB-eN6FPQDOzdRXCcSg01qhTMjuUQkBjLLBbolAC5UxfN-Mu5aARG4-8pxWM2q2CMI6xhR1a9XrrY595Q-enyClDuagi8mcyljXDvl6h2u1uIS1bVvKu3uu0ClLUHLYscdIUk7_gLli_BLUVtuQR0PbneL-IrxE57VrBuGFWNnnyO_LrB1s-tl3hPiFMgj2OuXSIl3KxHJhQJmI2gRDfam2TFR4vpuRSLzNJbsdca3mu4O5P8oAABqJfzAgTXPK2mExCUu-Z7BRzpINhMqqrEUln-W1w94yvCuaZ1o&recommendedFlavor=IN_NETWORK&refId=lC3fZ8%2BOil0oPKKL03koLA%3D%3D&trackingId=OQXA3luuV%2FVhjh6e%2Fpvi5g%3D%3D&trk=flagship3_search_srp_jobs,https://www.linkedin.com/jobs/view/3142889894/?eBP=CwEAAAGCoouzPYLgIGoLDtVAfof9LvO2P1alBqA1THYEyCCe55en_o0KYGXMxv6WbfqgVK_MgknSD1ijtBfjv3Urj2TL2b1fgVLWU--8eCr0HWpKgIOOgkmWFel4YWdJtgqztBlCp5i0pl15AEjfQLyAva8y5EV_MNtmpm_3zTsXM5_aXJcF9NOUFoU2RbCMutlQsPMHakTFM2kz3cZXtef5pAH8ipPUXHhJCufur9WyXqHLJ7qKrwoVsRT-AK34ph9bRUWU5dcNIbEHQbSNPaWr2G1tbQue8FP2W2gI1UX3I2zS5NmZJXaVG-E68Te7Sg3YWTfinyVMpZItIqS07W89DXI&recommendedFlavor=SCHOOL_RECRUIT&refId=FmgGMzrq0mbd8gIK9L1qXA%3D%3D&trackingId=bHJxzAGZqw8wzunOAl9LdQ%3D%3D&trk=flagship3_search_srp_jobs,https://www.linkedin.com/jobs/view/3216061552/?eBP=CwEAAAGCoouzPZgMMbS9U2-n9qUdecNHgMCgcxxWGJfpQwuGwbjDHaD0Y9I1bCfcESMyk3ilyB8SBAeGf3W5ukCC5UaZ4MrowXgqjohwYyAuhmMEJ-nPRMxISTuwokaOBRguFmDQ60yxeRhc_vSVXiufNIbxbO2PnEvsc-Ia9XizW7pvVfl4USGZeBk1A4Zz0i7hoy_gNI64-INwANk5m7ySb-8s0dwEJGhoCCupbZheu1WKVfbm19HCWosTIK8gEs-HAi8QGTIzKQQkBjdYY5V66dRd50CbYCBVyjlkFDSTp98O-xL4z7kqF3zRkj_h1vD0Dys8oKjExSKbw9s7NkuO&recommendedFlavor=SKILL_ASSESSMENTS&refId=FmgGMzrq0mbd8gIK9L1qXA%3D%3D&trackingId=APORR1NYtfLa8iCrgRTZ5g%3D%3D&trk=flagship3_search_srp_jobs,https://www.linkedin.com/jobs/view/3088349474/?eBP=CwEAAAGCoouzPUjMXyUcHkzhFQNYL_hmbdwV-VG51pozVjYfyCKveHGZu9j3N3UO2U4daBNFgZD3lVVL6-NAl6bJ1bcu5vB5q5ueL4kpcVXw0QpkJLsgjs5KQJCPiI3Kmqywtb8OPZ-Zzzcn57mAl1ziU-xoF2XVh9v3k2faG71KvDrCvP8CGoVCPQIzI71p-eN4Ee0QTllaWhdW_PYnWN9jH4YvKnDSkSQ9sQWY8W9jnfiQdWj2sw4-RQaxbjC3z7J_CdtxjgKYBJjYLQP8nsw80f6PLPTuHnaX4oJSpbO54SeCD-d6s6BmGZ1j6X2ld_OoYMIQth5JajIQF-gbx9rlc1quu15mfLnltHZgZRxtLx4oiZFWosv9p7KCAX5J&recommendedFlavor=IN_NETWORK&refId=FmgGMzrq0mbd8gIK9L1qXA%3D%3D&trackingId=lCmwULMHB%2FRHiN5N61%2Ft0A%3D%3D&trk=flagship3_search_srp_jobs,https://www.linkedin.com

In [156]:
# 1st run result dataframe - Checking null values
newdf.T.isnull().sum()

company          0
location         0
role             0
description    214
education      214
dtype: int64

In [ ]:
# Store data locally
newdf2.to_csv('jobs.csv')